# Line Integral Convolution

In [ ]:
import numpy as np

from scipy.ndimage import laplace

import matplotlib.pyplot as plt
from matplotlib.colors import Normalize
from matplotlib.cm import get_cmap

from PIL import Image

import volrender

Create the grid and some vector field

In [ ]:
nx = 400
ny = 200

xi = np.linspace(0, 10, nx + 1)
yi = np.linspace(0, 5, ny + 1)

x = 0.5 * (xi[1:] + xi[:-1])
y = 0.5 * (yi[1:] + yi[:-1])

X, Y = np.meshgrid(x, y, indexing='ij')
Xi, Yi = np.meshgrid(xi, yi, indexing='ij')

vortex_x = 5.0
vortex_y = 2.5

dx = X - vortex_x
dy = Y - vortex_y

r = np.sqrt(dx**2 + dy**2)
phi = np.arctan2(dy, dx)

VX = 1     - 5 * np.exp(- r**2 / 4) * np.sin(phi)
VY = Y / 5 + 5 * np.exp(- r**2 / 4) * np.cos(phi)

# define 2D vector field and its magnitude
vel = np.array([VX, VY]).transpose(1, 2, 0)
V = np.sqrt(VX**2 + VY**2)

# Now the LIC

In [ ]:
def contrast_enhance(data, sig=2.0):
    minval, maxval = data.mean() + sig * np.array([-1, 1]) * data.std()
    minval = max(0.0, minval)
    maxval = min(1.0, maxval)
    output = np.clip((data - minval) / (maxval - minval), 0.0, 1.0)
    return output

Make the noise

In [ ]:
noise = laplace(volrender.lic.gen_noise(nx, ny))

In [ ]:
noise_CE = contrast_enhance(noise)
noise_CE_LIC = volrender.lic.flic(noise_CE, x, y, vel)
noise_CE_LIC_LAPC = 0.1 + 0.8 * contrast_enhance(laplace(noise_CE_LIC))
noise_CE_LIC_LAP_LIC = volrender.lic.flic(noise_CE_LIC_LAPC, x, y, vel)
noise_CE_LIC_LAP_LIC_CE = 0.1 + 0.8 * contrast_enhance(noise_CE_LIC_LAP_LIC)
output = noise_CE_LIC_LAP_LIC_CE

imgs = [
    'noise',
    'noise_CE',
    'noise_CE_LIC',
    'noise_CE_LIC_LAPC',
    'noise_CE_LIC_LAP_LIC',
    'noise_CE_LIC_LAP_LIC_CE',
]
n = len(imgs)
f, ax = plt.subplots(n, 2, figsize=(8, 2 * n), dpi=100)

for i, name in enumerate(imgs):
    ax[i,0].imshow(locals()[name].T, cmap='gray', norm=Normalize(0, 1))
    ax[i,0].set_title(name)
    ax[i,0].set_axis_off()
    
    ax[i,1].hist(locals()[name].ravel(), fc='k')

Stitch LIC + Color together in HSV space

In [ ]:
# get the RGB colors for the two images
img_col = get_cmap('magma')(Normalize()(V))
img_lic = get_cmap('gray')(Normalize()(output))

# get numpy arrays of HSV values
hsv_col = np.array(Image.fromarray(np.uint8(img_col[:, :, :3] * 255)).convert('HSV'))
hsv_lic = np.array(Image.fromarray(np.uint8(img_lic[:, :, :3] * 255)).convert('HSV'))

# put together
hsv = hsv_col.copy()
hsv[..., -1] = np.uint8((np.float16(hsv_col[..., 2]) + np.float16(hsv_lic[..., 2]))/2)

# convert back to RGB-numpy array
rgb2 = np.array(Image.fromarray(hsv, mode='HSV').convert('RGB'))

In [ ]:
f, ax = plt.subplots(1, 3, figsize=(30, 5))
ax[0].imshow(V.T, cmap='magma', origin='lower')
ax[1].imshow(output.T, cmap='gray', origin='lower')
ax[2].imshow(rgb2.transpose(1,0,2), origin='lower')

# Compare two approaches
one is overlaying the images via the HSV color space, the other just somehow mixes the two matrices: LIC of the vectors and magnitude

In [ ]:
f, ax = plt.subplots(2, 1, sharex=True, sharey=True, figsize=(8, 8))

cc=ax[0].pcolormesh(Xi, Yi, rgb2[:, :, 0], facecolors=rgb2.transpose(0,1,2).reshape(-1, 3) / 255)
cc.set_array(None)
ax[0].set_title('via HSV')
ax[0].set_aspect('equal')

f = 0.25
cc=ax[1].pcolormesh(Xi, Yi, V * (1 - f + 2 * f * output), cmap='magma')
ax[1].set_title('matrix product')
ax[1].set_aspect('equal')